In [1]:
import pandas as pd
import geopandas as gp
import folium
import numpy as np

In [2]:
year = 2018

In [6]:
acs = pd.read_csv('D:/TNC-Demand-Model/Outputs/Chicago_ACS_DATA.csv')

In [9]:
tracts = gp.read_file('D:/TNC-Demand-Model/Inputs/Census Shapefiles/Illinois/Chicago Tracts/geo_export_558aad9f-98d8-4dd5-a6b1-c1730155d596.shp')

In [4]:
tracts.geoid10

NameError: name 'tracts' is not defined

In [10]:
acs['CENSUS_TRACT'] = acs.CENSUS_TRACT.astype(str)

In [13]:
geo_area = gp.read_file('D:/TNC-Demand-Model/Inputs/Census Shapefiles/Illinois/Tracts/tl_2019_17_tract.shp')

geo_area['SQUARE_MILES_LAND'] = geo_area['ALAND']*3.86102e-7
geo_area['GEOID'] = geo_area.GEOID.astype(str)

acs = acs.merge(geo_area[['GEOID','SQUARE_MILES_LAND']], how = 'left', left_on = 'CENSUS_TRACT', right_on = 'GEOID')

In [14]:
acs['TOTAL_POP_PER_SQ_MILE'] = acs['TOTAL_POP'] / acs['SQUARE_MILES_LAND']
acs['TOTAL_HHLDS_PER_SQ_MILE'] = acs['TOTAL_HHLDS'] / acs['SQUARE_MILES_LAND']

In [15]:
normalize_vars = ['AGE_25_34', 'TOTAL_POP_25P_BACHELORS_HIGHER','TOTAL_HHLDS_200KP' ]

for var in normalize_vars:
    acs[var] = acs[var].replace('-',np.nan)
    acs[var] = acs[var].replace('250,000+','250000')
    acs[var] = acs[var].astype(float)
    acs[var + '_PER_POP'] = acs[var]/acs['TOTAL_POP']

In [17]:
centroids = pd.read_csv('D:/TNC-Demand-Model/otp/points.csv')

centroids = gp.GeoDataFrame(centroids)

In [18]:
acs['HHLDS_0_VEH_PER_HHLD'] = acs['HHLDS_0_VEH']/acs['TOTAL_HHLDS']

In [19]:
acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].replace('-',np.nan)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].replace('-',np.nan)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].replace('-',np.nan)

acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].astype(float)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].astype(float)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].astype(float)


acs['HHLDS_100KP_PER_HHLD'] = (acs['TOTAL_HHLDS_100K_150K'] +acs['TOTAL_HHLDS_150K_200K'] + acs['TOTAL_HHLDS_200KP'])/acs['TOTAL_HHLDS'] 

In [20]:
variables = acs.columns.drop(['Unnamed: 0', 'CENSUS_TRACT', 'GEOID'])

In [28]:
variables = ['HHLDS_MEAN_INCOME']

In [21]:
acs['TOTAL_HHLDS_100K_150K'] = acs['TOTAL_HHLDS_100K_150K'].astype(float)
acs['TOTAL_HHLDS_150K_200K'] = acs['TOTAL_HHLDS_150K_200K'].astype(float)
acs['TOTAL_HHLDS_200KP'] = acs['TOTAL_HHLDS_200KP'].astype(float)


In [22]:
acs[acs['SQUARE_MILES_LAND'] > 10].GEOID.astype(str)

780       17031804201
1081      17031823801
2099      17031804201
2400      17031823801
3418      17031804201
             ...     
102644    17031823801
103662    17031804201
103963    17031823801
104981    17031804201
105282    17031823801
Name: GEOID, Length: 160, dtype: object

In [34]:
for var in variables:
    acs[var] = acs[var].replace('-',np.nan)
    acs[var] = acs[var].replace('250,000+','250000')

    acs[var] = acs[var].astype(float)
    print('Working on variable ' + var)
    m = folium.Map([41.8781, -87.6298], zoom_start=11)
    print(acs[var].max())
    print(acs[var].min())
    print(acs[var].mean())

    bins = np.nanquantile(acs[var], [0, 0.8,0.99,1])
    print(bins)
    
    # Add the color for the chloropleth:
    m.choropleth(
     geo_data=tracts,
     name= var,
     data=acs,
     columns = ['CENSUS_TRACT', var],
     key_on='feature.properties.geoid10',
     fill_color='BuGn',
     fill_opacity=0.6,
     line_opacity=0.2,
     bins = bins,
     legend_name= var + ' Totals',
     highlight = True
    )

    feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)

    for tract2 in centroids.GEOID:
        row = centroids[centroids['GEOID'] == tract2]
        folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)
    
    m.add_child(feature_group)
    
    folium.LayerControl().add_to(m)

    m.save('D:/TNC-Demand-Model/Data Exploration/ACS Maps/' + str(year) + '/' + var + '_Map.html')


Working on variable HHLDS_MEAN_INCOME
466161.0
18015.0
90198.44885931558
[ 18015. 119059. 265489. 466161.]


C:\Users\transportlab\anaconda3\lib\site-packages\folium\folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning
